In [1]:
import pandas as pd
import altair as alt
from fdp.db import query

from IPython.display import Markdown

_ = alt.themes.enable("powerbi")


In [2]:
df = query(
    """
    select
        date_trunc('day', sector_start_at) as date,
        sum(piece_size_tibs) as onboarded_tibs
    from local.filecoin_state_market_deals
    where
        sector_start_epoch > 0 and slash_epoch = -1
        and sector_start_at < current_date
        and sector_start_at > '2022-01-01'
    group by 1
    order by 1
"""
)
df["date"] = pd.to_datetime(df["date"])



::: {.callout-note}
This dashboard is using data from  [Glif nodes](https://lotus.filecoin.io/lotus/developers/glif-nodes/) `StateMarketDeals` periodic dump on S3 ([direct link](https://marketdeals.s3.amazonaws.com/StateMarketDeals.json.zst)).
:::

## Data Onboarding Rate

In [3]:
# | output: true

c = (
    alt.Chart(df)
    .mark_bar()
    .encode(
        x=alt.X("date:T", title="Date"),
        y=alt.Y("onboarded_tibs:Q", title="Onboarded TiBs"),
        tooltip=[
            alt.Tooltip("date:T", title="Date"),
            alt.Tooltip("onboarded_tibs:Q", title="Onboarded TiBs"),
        ],
    )
    .properties(width="container", height=400)
    # .interactive(bind_y=False)
)

c.display()


alt.Chart(...)

### Cumulative Data Onboarded

In [4]:
# | output: true

total_data_onboarded = df["onboarded_tibs"].sum()
df["cumulative_data"] = df["onboarded_tibs"].cumsum()

alt.Chart(df).mark_area().encode(
    x=alt.X("date:T", title="Date"),
    y=alt.Y("cumulative_data:Q", title="Cumulative Data Onboarded (PiB)"),
    tooltip=[
        alt.Tooltip("date:T", title="Date"),
        alt.Tooltip("cumulative_data:Q", title="Cumulative Data Onboarded (PiB)"),
    ],
).properties(
    width="container",
    height=400,
    title=f"{total_data_onboarded:.2f} TiBs Onboarded",
).configure_title(fontSize=32)


alt.Chart(...)

### Onboarding Growth Rate (7 Day Rolling Average)

In [5]:
# | output: true

df["growth_rate"] = df["onboarded_tibs"].pct_change(periods=7).rolling(7).mean() * 100
recent_df = df[df["date"] > "2022-01-01"]

alt.Chart(recent_df).mark_bar().encode(
    x=alt.X("date:T", title="Date"),
    y=alt.Y("growth_rate:Q", title="Onboarding Growth Rate"),
    tooltip=[
        alt.Tooltip("date:T", title="Date"),
        alt.Tooltip("growth_rate:Q", title="Onboarding Growth Rate"),
    ],
).properties(
    width="container",
    height=400,
)


alt.Chart(...)